In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings;
from pysqldf import SQLDF
import pandasql as psql
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import KFold
import sklearn.ensemble as ske
import lightgbm as lgb
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

warnings.filterwarnings('ignore')

In [2]:
train01 = pd.read_csv("C:\\Kaggle\\Cars\\Data\\TrnDataForLGB.csv")
test01 = pd.read_csv("C:\\Kaggle\\Cars\\Data\\TstDataForLGB.csv")

Remove_List = ["id","Price","Name","Lag_Price2_MIN","Lag_Price2_MAX",
               "Lag_Price3_MIN","Lag_Price3_MAX","Lag_Price","Lag_Price3","Engine_Group",
               "Power_Group","TrainTestInd","CarCompName","RateChng1","RateChng2","RateChng3",
               "Lag_Price4_MIN","Lag_Price4_MAX","Lag_Price4_MIN_BY_MAX"]
feature_names = list(set(list(train01.columns)) - set(Remove_List))

In [3]:
train01[feature_names].columns[train01[feature_names].isnull().any()]

Index(['New_Price', 'Lag_Price4', 'Lag_Price2'], dtype='object')

In [4]:
test01[feature_names].columns[test01[feature_names].isnull().any()]

Index(['New_Price', 'Lag_Price4', 'Lag_Price2'], dtype='object')

In [5]:
train01['Lag_Price4'] = train01['Lag_Price4'].fillna(value=0)
train01['Lag_Price2'] = train01['Lag_Price2'].fillna(value=0)
train01['New_Price'] = train01['New_Price'].fillna(value=0)

test01['Lag_Price4'] = test01['Lag_Price4'].fillna(value=0)
test01['Lag_Price2'] = test01['Lag_Price2'].fillna(value=0)
test01['New_Price'] = test01['New_Price'].fillna(value=0)

In [6]:
print(train01[feature_names].columns[train01[feature_names].isnull().any()])
print(test01[feature_names].columns[test01[feature_names].isnull().any()])

Index([], dtype='object')
Index([], dtype='object')


In [7]:
x_train02 = train01[feature_names]
y_train02 = train01['Price']
x_sub2 = test01[feature_names]

In [8]:
train_stats = x_train02.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Power,6019.0,112.668894,53.940547,34.2,74.00,93.70,138.1,560.00
LocationChennai,6019.0,0.082073,0.274499,0.0,0.00,0.00,0.0,1.00
Owner_TypeFourth...Above,6019.0,0.001495,0.038643,0.0,0.00,0.00,0.0,1.00
CompNameCarName_CAMRY,6019.0,0.001828,0.042714,0.0,0.00,0.00,0.0,1.00
CompName_NISSAN,6019.0,0.015119,0.122036,0.0,0.00,0.00,0.0,1.00
CompName_BMW,6019.0,0.044360,0.205910,0.0,0.00,0.00,0.0,1.00
CompNameCarName_ALTO,6019.0,0.023924,0.152826,0.0,0.00,0.00,0.0,1.00
CompNameCarName_CLA,6019.0,0.052500,0.223052,0.0,0.00,0.00,0.0,1.00
CompNameCarName_SCORPIO,6019.0,0.010135,0.100168,0.0,0.00,0.00,0.0,1.00
CompNameCarName_TERRANO,6019.0,0.004320,0.065587,0.0,0.00,0.00,0.0,1.00


In [9]:
def norm(x):
  return(x - train_stats['mean']) / train_stats['std']
normed_x_train02 = norm(x_train02)
normed_x_sub2 = norm(x_sub2)

In [10]:
normed_x_train02.reset_index(drop = True, inplace = True)
kf = KFold(n_splits = 5, shuffle = True, random_state = 100)
kf.get_n_splits(normed_x_train02)

5

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU
from keras import optimizers
from keras.callbacks import EarlyStopping
import math

def nn_model():
    model = Sequential()
    model.add(Dense(400, input_dim = normed_x_train02.shape[1], init = 'he_normal'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(400, init = 'he_normal'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(200, init = 'he_normal'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.4))
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss = 'mean_squared_error', optimizer = 'Adam')
    return(model)

In [32]:
IterationNum = 1
for train_index, test_index in kf.split(normed_x_train02):
    print("Running CV Iteration Num :", IterationNum)
    MOD_DATA_2_TRAIN, MOD_DATA_2_TEST = train01.iloc[train_index], train01.iloc[test_index]
    X_TRAIN, X_TEST = normed_x_train02.iloc[train_index], normed_x_train02.iloc[test_index]
    Y_TRAIN, Y_TEST = y_train02[train_index], y_train02[test_index]
    
    model = nn_model()
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    fin_model = model.fit(X_TRAIN, Y_TRAIN, validation_data = (X_TEST, Y_TEST), epochs = 200, batch_size = 32, verbose = 1, callbacks=[es])
    
    MOD_DATA_2_TEST['Predicted_Model_Value'] = model.predict(X_TEST)[:,0]
    if(IterationNum == 1):
        CV_SCORED_DATA = MOD_DATA_2_TEST.copy(deep=True)
        CV_SCORED_DATA.reset_index(drop = True, inplace = True)
        test_scored = model.predict(normed_x_sub2)[:,0]
    else:
        CV_SCORED_DATA = pd.concat([CV_SCORED_DATA,MOD_DATA_2_TEST])
        CV_SCORED_DATA.reset_index(drop = True, inplace = True)
        test_scored = test_scored + model.predict(normed_x_sub2)[:,0]
                    
    IterationNum = IterationNum + 1
    
    print("Test RMSE : ",sqrt(mean_squared_error(MOD_DATA_2_TEST["Price"], MOD_DATA_2_TEST['Predicted_Model_Value'])))
    
print("CV RMSE : ",sqrt(mean_squared_error(CV_SCORED_DATA["Price"], CV_SCORED_DATA['Predicted_Model_Value'])))
#CV RMSE :  0.15639046727479403
#LB: 0.9387

Running CV Iteration Num : 1
Train on 4815 samples, validate on 1204 samples
Epoch 1/200
4815/4815 [==============================] - 4s 787us/step - loss: 0.9497 - val_loss: 0.0978
Epoch 2/200
4815/4815 [==============================] - 2s 361us/step - loss: 0.2295 - val_loss: 0.0567
Epoch 3/200
4815/4815 [==============================] - 2s 378us/step - loss: 0.1588 - val_loss: 0.0467
Epoch 4/200
4815/4815 [==============================] - 2s 372us/step - loss: 0.1344 - val_loss: 0.0384
Epoch 5/200
4815/4815 [==============================] - ETA: 0s - loss: 0.114 - 2s 353us/step - loss: 0.1145 - val_loss: 0.0359
Epoch 6/200
4815/4815 [==============================] - 2s 346us/step - loss: 0.1074 - val_loss: 0.0299
Epoch 7/200
4815/4815 [==============================] - 2s 351us/step - loss: 0.1005 - val_loss: 0.0292
Epoch 8/200
4815/4815 [==============================] - 2s 357us/step - loss: 0.0954 - val_loss: 0.0285
Epoch 9/200
4815/4815 [==============================] - 2s

Epoch 33/200
4815/4815 [==============================] - 2s 353us/step - loss: 0.0448 - val_loss: 0.0243
Epoch 34/200
4815/4815 [==============================] - 2s 357us/step - loss: 0.0441 - val_loss: 0.0288
Epoch 35/200
4815/4815 [==============================] - 2s 359us/step - loss: 0.0441 - val_loss: 0.0245
Epoch 36/200
4815/4815 [==============================] - 2s 362us/step - loss: 0.0425 - val_loss: 0.0302
Epoch 37/200
4815/4815 [==============================] - 2s 357us/step - loss: 0.0407 - val_loss: 0.0308
Epoch 38/200
4815/4815 [==============================] - 2s 355us/step - loss: 0.0405 - val_loss: 0.0245
Epoch 39/200
4815/4815 [==============================] - 2s 360us/step - loss: 0.0395 - val_loss: 0.0236
Epoch 00039: early stopping
Test RMSE :  0.15371392636132974
Running CV Iteration Num : 3
Train on 4815 samples, validate on 1204 samples
Epoch 1/200
4815/4815 [==============================] - 4s 778us/step - loss: 0.8769 - val_loss: 0.0946
Epoch 2/200
481

4815/4815 [==============================] - 2s 351us/step - loss: 0.0265 - val_loss: 0.0323
Epoch 71/200
4815/4815 [==============================] - 2s 361us/step - loss: 0.0273 - val_loss: 0.0367
Epoch 72/200
4815/4815 [==============================] - 2s 366us/step - loss: 0.0279 - val_loss: 0.0402
Epoch 73/200
4815/4815 [==============================] - 2s 375us/step - loss: 0.0282 - val_loss: 0.0344
Epoch 74/200
4815/4815 [==============================] - 2s 374us/step - loss: 0.0283 - val_loss: 0.0333
Epoch 75/200
4815/4815 [==============================] - 2s 367us/step - loss: 0.0264 - val_loss: 0.0336
Epoch 76/200
4815/4815 [==============================] - 2s 374us/step - loss: 0.0257 - val_loss: 0.0330
Epoch 77/200
4815/4815 [==============================] - 2s 372us/step - loss: 0.0258 - val_loss: 0.0339
Epoch 78/200
4815/4815 [==============================] - 2s 374us/step - loss: 0.0242 - val_loss: 0.0324
Epoch 79/200
4815/4815 [==============================] - 2

Epoch 56/200
4815/4815 [==============================] - 2s 363us/step - loss: 0.0326 - val_loss: 0.0248
Epoch 57/200
4815/4815 [==============================] - 2s 359us/step - loss: 0.0317 - val_loss: 0.0226
Epoch 58/200
4815/4815 [==============================] - 2s 355us/step - loss: 0.0321 - val_loss: 0.0242
Epoch 59/200
4815/4815 [==============================] - 2s 361us/step - loss: 0.0324 - val_loss: 0.0227
Epoch 60/200
4815/4815 [==============================] - 2s 368us/step - loss: 0.0300 - val_loss: 0.0242
Epoch 61/200
4815/4815 [==============================] - 2s 369us/step - loss: 0.0307 - val_loss: 0.0231
Epoch 62/200
4815/4815 [==============================] - 2s 370us/step - loss: 0.0307 - val_loss: 0.0234
Epoch 63/200
4815/4815 [==============================] - 2s 369us/step - loss: 0.0298 - val_loss: 0.0224
Epoch 64/200
4815/4815 [==============================] - 2s 382us/step - loss: 0.0286 - val_loss: 0.0223
Epoch 65/200
4815/4815 [======================

In [33]:
from scipy import stats
test_scored2 = test_scored / 5.0
stats.describe(test_scored2)

DescribeResult(nobs=1234, minmax=(0.73964167, 4.1613693), mean=1.9749265, variance=0.490959, skewness=0.7544870376586914, kurtosis=0.08592567944426888)

In [35]:
test_scored3 = pd.DataFrame({'Price' : np.exp(test_scored2)-1})
test_scored3.head()

,Price
0,2.607597
1,2.792425
2,16.253904
3,4.002155
4,4.369020


In [36]:
CV_SCORED_DATA.to_csv("C:\\Kaggle\\Cars\\CV_Scored\\20190716_Keras01copy_CVTRAIN_DS.csv",
                      index = False)
test_scored3.to_csv("C:\\Kaggle\\Cars\\Submission\\20190716_Keras01copy_TEST_DS.csv",
                    index = False)